In [72]:
!uv add beautifulsoup4 requests tqdm

Resolved 98 packages in 193ms                                        
Installed 1 package in 20ms                                      
 + tqdm==4.67.1


In [3]:
import requests
from pprint import pprint
from dataclasses import dataclass, asdict
from typing import List
from bs4 import BeautifulSoup
from tqdm import tqdm

api_key = "0iVKSDmfcvLekS1QjsJRqpUaztocZ2eSFJD5gi29"
params = {
    "api_key": api_key,
    "format": "json",
    "limit": 50
}
url = "https://api.congress.gov/v3/law/119"

@dataclass
class Law:
    title: str
    sponsor: List[str]
    actions: List[dict]
    text: str
    originChamberCode: str
    congress: int
    number: int

    def to_json(self) -> str:
        import json
        return json.dumps(asdict(self), indent=2)
        
# Get all laws
bills = requests.get(url, params=params).json().get('bills')
# Get a bill
for bill_0 in tqdm(bills):
    bill = requests.get(bill_0.get('url'), params=params).json().get("bill")
    title = bill.get('title')
    sponsors = bill.get('sponsors')[0]
    originChamberCode = bill.get('originChamberCode')

    # Get member info
    member = requests.get(sponsors.get('url'), params=params).json().get('member')
    officialWebsiteUrl = member.get("officialWebsiteUrl")
    # Get most recent term first
    terms = member.get('terms')
    terms.reverse()
    
    state = member.get('state')
    name = member.get('directOrderName')
    bioguideId = member.get('bioguideId')
    
    sponsor = {
        'name': name,
        'websiteUrl': officialWebsiteUrl,
        'terms': terms,
        'state': state,
        'bioguideId': bioguideId
    }
    
    number = bill.get('number')
    congress = bill.get('congress')
    
    # Get Bill actions
    actions = requests.get(bill.get('actions').get('url'), params=params).json().get('actions')
    actions.reverse()
    
    # Get Bill Text
    text = requests.get(bill.get('textVersions').get('url'), params=params).json()
    formats = text.get('textVersions')[0].get('formats')
    text = next((item for item in formats if item['type'] == 'Formatted Text'), None)
    bill_text_raw = requests.get(text.get('url'))
    soup = BeautifulSoup(bill_text_raw.text, "html.parser")
    text = soup.find('pre').get_text().strip()
    law = Law(title, sponsor, actions, text, originChamberCode, congress, number)

    with open(f'data/{originChamberCode}.{congress}.{number}.json', 'w') as f:
        f.write(law.to_json())


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:54<00:00,  1.52s/it]


In [74]:
# member data

sponsor = requests.get('https://api.congress.gov/v3/member/C001056?format=json', params=params).json()
sponsor.get("officialWebsiteUrl")

{'member': {'addressInformation': {'city': 'Washington',
   'district': 'DC',
   'zipCode': 20510},
  'bioguideId': 'C001056',
  'birthYear': '1952',
  'cosponsoredLegislation': {'count': 3939,
   'url': 'https://api.congress.gov/v3/member/C001056/cosponsored-legislation'},
  'currentMember': True,
  'depiction': {'attribution': '<a href="http://www.senate.gov/artandhistory/history/common/generic/Photo_Collection_of_the_Senate_Historical_Office.htm">Courtesy U.S. Senate Historical Office</a>',
   'imageUrl': 'https://www.congress.gov/img/member/c001056_200.jpg'},
  'directOrderName': 'John Cornyn',
  'firstName': 'John',
  'honorificName': 'Mr.',
  'invertedOrderName': 'Cornyn, John',
  'lastName': 'Cornyn',
  'leadership': [{'congress': 113, 'type': 'Assistant Democratic Leader'},
   {'congress': 114, 'type': 'Assistant Majority Leader'},
   {'congress': 115, 'type': 'Majority Whip'}],
  'officialWebsiteUrl': 'https://www.cornyn.senate.gov/',
  'partyHistory': [{'partyAbbreviation': '